In [1]:
import pyvista as pv
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import sys
np.random.seed(0)

In [2]:
# import mesh
filename = 'inputs/nut.ply'
# filename = 'inputs/sphere.ply'
# filename = 'inputs/cow.vtp'
# filename = 'panther.stl'
mesh = pv.read(filename)


# mesh = mesh.triangulate()
# mesh = mesh.decimate(0.7)
# mesh = mesh.triangulate()

# stretch points along z for ellipse
# mesh.points[:,2] *= 20

pl = pv.Plotter()
_ = pl.add_mesh(mesh, show_edges=True)
_ = pl.add_axes_at_origin(ylabel=None)
pl.camera_position = 'xz'
pl.show()

Widget(value='<iframe src="http://localhost:38897/index.html?ui=P_0x7f802e14efe0_0&reconnect=auto" class="pyvi…

In [3]:
# forward problem with growth

# forward problem params
nx = mesh.n_points
niter = 1000
dt = 0.0001
# dx = .1 # good for decimated cow
dx = .025 # good for decimated (or non-decimated) nut
# dx = .002/2 # good for decimated sphere

In [4]:
# solve simple inverse problem for growth
# to end with mesh starting from sphere

# initialize points
pts = np.zeros((nx, 3, niter+1))
pts[:,:,0] = mesh.points.copy()


# get each point's distance from origin
dist = np.sqrt(np.sum(pts[:,:,0]**2, axis=1)) 

# construct final points: a sphere, by shrinking by distance factors
pts[:,:,-1] = np.einsum('ui,u->ui', pts[:,:,0], 1/dist)

# make final points partway between sphere and full shape to avoid singularities
pts[:,:,-1] = 0.5*pts[:,:,0] + 0.5*pts[:,:,-1]

# interpolate points in between
for i in range(1, niter):
    pts[:,:,i] = (1-i/niter) * pts[:,:,0] + i/niter * pts[:,:,-1]
    # print(i/(niter-1))
    # print(i)
    
# reverse time ordering to go from shrinking to growing
pts_flipped = np.flip(pts, axis=2)

# write result to disk
np.save('pts.npy', pts_flipped)

In [5]:
# growth
# updates mesh points and calculates laplacians
from mesh_laplacian import compute_mesh_laplacian

grow = False # whether to calculate a new laplacian at each point in time
            # or just copy them all from initial point


grow_from_rule = False # True = grow via assigned growth rule
                      # False = read growth from file

if grow:
    
    if grow_from_rule:
        # growth params
        growth_rate = 1.001

        # initialize points 
        pts = np.zeros((nx, 3, niter+1))
        pts[:,:,0] = mesh.points.copy()

    else:
        # reads points from file
        pts = np.load('pts.npy')

    # initialize laplacians
    laps = np.zeros((nx, nx, niter+1))
    laps[:,:,0] = compute_mesh_laplacian(mesh)


    # run growth loop
    print("Beginning growth loop...")
    for i in range(niter):
        sys.stdout.write("\rIteration {0}/{1} ({2}%)".format(i+1, niter, int(100*(i+1)/niter)))
        sys.stdout.flush()

        if grow_from_rule:
            # update points according to growth rule
            # pts[:,:,i+1] = pts[:,:,i] * np.random.uniform(1.0, growth_rate, (nx, 3)) # randomly growing isotropically

            # pts[:,:,i+1] = pts[:,:,i] 
            # pts[:,2,i+1] = pts[:,2,i] *  np.random.uniform(1.0, growth_rate, nx) # randomly growing in z direction 
            
            pts = pts[:,:,0] * growth_rate * dt * i # uniform additive isotropic growth

        mesh.points = pts[:,:,i+1]

        # calculate laplacian
        laps[:,:,i+1] = compute_mesh_laplacian(mesh)

    # reset mesh points
    mesh.points = pts[:,:,0]
    
else:
    # initialize points 
    pts = np.zeros((nx, 3, niter+1))
    pts[:,:,0] = mesh.points.copy()
    
    # copy initial points to all times
    pts = np.stack((pts[:,:,0],) * (niter+1), axis=2)

        
    # initialize laplacians
    laps = np.zeros((nx, nx, niter+1))
    laps[:,:,0] = compute_mesh_laplacian(mesh)

    # copy initial laplacian to all times
    laps = np.stack((laps[:,:,0],) * (niter+1), axis=2)

print("\nGrowth loop completed.")

# write results to disk
np.save('laps.npy', laps)


Growth loop completed.


In [6]:
# RD integration
from forward import step_se

# RD params
du = 1
dv = 10
g = 1000 # set to zero to just test diffusion
a = 0.126779*0.25
b = 1.1

# initialize fields near steady-state solution
u = np.ones(nx, dtype=float)*(a+b)
u += np.random.normal(scale=0.01, size=nx)
v = np.ones(nx, dtype=float)*(b/(a+b)**2)

u_stored = np.zeros((nx, niter+1))
u_stored[:,0] = u

v_stored = np.zeros((nx, niter+1))
v_stored[:,0] = v


integrate = True

if integrate:
    print("Beginning RD integration loop...")
    for i in range(niter):
        sys.stdout.write("\rIteration {0}/{1} ({2}%)".format(i+1, niter, int(100*(i+1)/niter)))
        sys.stdout.flush()

        # Run GMRES to solve for next timestep
        # reference calculated laplacians from growth loop
        u, v = step_se(u,v, a,b,g,du,dv, laps[:,:,i], dx,nx,dt)

        # store for later animation
        u_stored[:,i+1] = u
        v_stored[:,i+1] = v


    print("\nRD loop completed.")
   
# write solution onto mesh
mesh.point_data['u'] = u
mesh.point_data['v'] = v

Beginning RD integration loop...
Iteration 1000/1000 (100%)
RD loop completed.


In [7]:
# run plotting 
# pts = np.load('pts.npy')

# Set up plotting
plotting = 'static'
# plotting = 'dynamic'
skip = 25

if plotting == 'static':
    p = pv.Plotter(shape=(1,4), notebook=0)
    mesh.point_data['u'] = u
    p.add_mesh(mesh.copy(), scalars='u', cmap='gray')
    
    def plot_mesh(u, subplot):
        mesh.point_data['u'] = u
        p.subplot(0,subplot)
        p.add_mesh(mesh.copy(), scalars='u', cmap='gray', show_edges=True)
        p.link_views()
        p.view_isometric()
        
    plot_mesh(u_stored[:,0], 0)
   
elif plotting == 'dynamic':
    mesh.points = pts[:,:,-1]
    plotter = pv.Plotter(notebook=False, off_screen=True)
    plotter.add_mesh(
        mesh.rotate_y(-45),
        scalars=u,
        lighting=False,
        show_edges=True,
        scalar_bar_args={"title": "u"},
        clim=[u_stored.min(), u_stored.max()],
        cmap='hot'
    )
    plotter.camera_position = 'xy'

    # Open a gif
    plotter.open_gif("nut.gif")
    # plotter.camera.zoom(0.8) # have to zoom out to accomodate growth
                             # would be nice if I could get around this by plotting final mesh first
        


print("Beginning plotting loop...")
for i in range(niter):
    sys.stdout.write("\rIteration {0}/{1} ({2}%)".format(i+1, niter, int(100*(i+1)/niter)))
    sys.stdout.flush()
    if plotting == 'static':
        if i==int(niter/3):
            # mesh.points = pts[:,:,i]
            plot_mesh(u_stored[:,i], 1)

        if i==int(niter*2/3):
            # mesh.points = pts[:,:,i]
            plot_mesh(u_stored[:,i], 2)

        if i==niter-1:
            # mesh.points = pts[:,:,i]
            plot_mesh(u_stored[:,i], 3)
            
    elif plotting == 'dynamic':
        if i%skip==0:
            plotter.update_coordinates(pts[:,:,i], render=False)
            plotter.update_scalars(u_stored[:,i], render=False)

            # Write a frame. This triggers a render.
            plotter.write_frame()

print("\nPlotting loop completed.")
      


# p = pv.Plotter(shape=(1,1), notebook=0)
# mesh.point_data['turing'] = u
# p.add_mesh(mesh, scalars='turing', cmap='gray')      
if plotting == 'static': p.show()
elif plotting == 'dynamic': plotter.close()

print("Plotting completed.")
# plt.show()
# print(u, info)

Beginning plotting loop...
Iteration 1000/1000 (100%)
Plotting loop completed.
Plotting completed.


Computing laplacian at each step is way too slow. But I don't see a way around it.

Might be best to run growth and RD separately: first do growth and calculate laplacians, then do RD and reference calculated laplacians. Then I can also separate the animation loop.

Example growth rule: sphere into cow

Get rule by first shrinking cow into sphere.

In [8]:
# create new mesh with these points to visualize
new_mesh = mesh.copy()
new_mesh.points = pts_flipped[:,:,10]

pl = pv.Plotter()
_ = pl.add_mesh(new_mesh, show_edges=True)
_ = pl.add_axes_at_origin(ylabel=None)
pl.camera_position = 'xy'
pl.show()

Widget(value='<iframe src="http://localhost:38897/index.html?ui=P_0x7f7f8e3fd6c0_1&reconnect=auto" class="pyvi…

In [9]:
a = np.array([1,2,3])

result = np.stack((a,) * 4, axis=1)

result

array([[1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3]])

In [10]:
pts[0,0,:]

array([71.15364838, 71.15364838, 71.15364838, ..., 71.15364838,
       71.15364838, 71.15364838])

<img src="growth_test.gif" width="250" align="center">